### Operacionalização e Teste das Hipóteses
--------------

Esse notebook foi usado para gerar os resultados que estão demonstrados na documentação.pdf

In [1]:
import pickle
import pprint

with open('./Pickle/palavras_pvalue.pickle', 'rb') as handle:
    dic_pvalue = pickle.load(handle)

In [2]:
def get_fracao(value, size):
    l = []
    for n in range(size):
        l.append(value[n][2]/value[n][1])
        a = value[n][2]
        b = value[n][1]
        print(a, b, a/b)
        
    return l

In [14]:
import pandas as pd

dts = {}
for key, value in dic_pvalue.items():
    print(key)
    size = len(value)

    palavra, P_total, P_estado, P_value, Nt, Ne = zip(*value)
    dt = pd.DataFrame({
        'palavra' : palavra,
        'P_total' : P_total,
        'P_estado': P_estado,
        'P-value' : P_value,
        #'Fração'  : fracao
        'Fração' : [P_estado[n] / P_total[n] for n in range(size)]
        })
    dts[key] = dt


Teresina
Aracaju
Macapá
Salvador
Belo Horizonte
Goiânia
Recife
Manaus
Fortaleza
Porto Alegre
Belém
João Pessoa
Natal
Rio Branco
São Paulo
Porto Velho
Campo Grande
Boa Vista
Brasília
Cuiabá
Curitiba
Maceió
Palmas
Florianópolis
Vitória
São Luís
Rio de Janeiro


In [15]:
sp = dts['São Paulo']
po = dts['Porto Alegre']
na = dts['Natal']
ri = dts['Rio Branco']
pv = dts['Porto Velho']
ca = dts['Campo Grande'] 
bo = dts['Boa Vista'] 
br = dts['Brasília'] 
pa = dts['Palmas']
fl = dts['Florianópolis']
vi = dts['Vitória']
sl = dts['São Luís'] 
rj = dts['Rio de Janeiro']

In [16]:
#dt['foo'] = (dt['Nt'] - dt['Ne'])/(dt['Ne'])

In [23]:
#dt[dt['P-value'] < 0.05].sort_values(by=['Fração'], ascending=False)

In [69]:
sp[sp['P-value']<0.05].sort_values(by=['Fração'], ascending=False)[:10]

,palavra,P_total,P_estado,P-value,Fração
7634,',0.000308,0.000117,1.374083e-02,0.379525
2883,►,0.001158,0.000592,2.220446e-16,0.511442
1165,Ver,0.000339,0.000203,3.428226e-02,0.597132
6086,component,0.000323,0.000194,4.490833e-02,0.599998
1858,t,0.000584,0.000353,3.272612e-04,0.604505
4450,2017,0.000414,0.000260,1.579257e-02,0.629874
3538,2016,0.000350,0.000226,4.880847e-02,0.646051
9269,R,0.001359,0.000888,5.040413e-14,0.653092
672,00,0.000622,0.000407,5.911799e-04,0.654114
6061,5,0.000619,0.000410,8.207331e-04,0.663142


In [28]:
be[be['P-value']<0.05].sort_values(by=['Fração'], ascending=True)[:10]

,palavra,P_total,P_estado,P-value,Fração
2990,p,0.000602,0.000041,0.045483,0.068753
2984,c,0.000771,0.000066,0.004992,0.085643
651,b,0.000720,0.000074,0.004871,0.102575
807,►,0.001158,0.000120,0.000006,0.103256
924,d,0.000563,0.000064,0.022491,0.112978
993,var,0.000508,0.000090,0.017008,0.176906
2230,function,0.000695,0.000135,0.000826,0.193715
1032,"""",0.000884,0.000188,0.000013,0.213205
2458,i,0.000501,0.000126,0.011016,0.250868
499,t,0.000584,0.000155,0.002794,0.264884
